In [ ]:
import datetime
now = datetime.datetime.now()
print(now)
nowDate = now.strftime('%Y-%m-%d')
print(nowDate)

In [ ]:
folder = nowDate.replace('-','/')
print(folder)
folder = "2017/09/27"
print(folder)

In [ ]:
from webCrawler import *

In [ ]:
clickTabOfFireFox(4)
clickEmpty(119/300)
moveScroll(20)
clickHattrickLogin()
clickMyClub()
clickEmpty(183/300)
moveScroll(20)
clickPlayer()
clickEmpty(155/300)
moveScroll(20)
clickSkillTable()
moveScroll(-5)
clickSince(318/380) # 선수 24명 = 311/380
clickCopy()
clickClose()
moveScroll(20)
clickSkillTable()
clickHattrickLogout()
clickURL()
clickTabOfFireFox(7)

In [ ]:
import pyperclip # The name you have the file
tmpClip = pyperclip.paste()
print(tmpClip)

In [ ]:
import os.path
filePath = folder + '/player.html'

if not os.path.exists(folder):
    print(folder + "\t\t : will be created")
    os.makedirs(folder)
else :
    print(folder + "\t\t : is existed")

if ( os.path.exists(filePath) ):
    print(filePath + "\t : will be removed")
    os.remove(filePath)
    
if not os.path.exists(filePath):
    print(filePath + "\t : is not exists")

In [ ]:
with open(folder+'/player.html', 'a') as f:
    f.write(tmpClip)

In [ ]:
from parser import *

In [ ]:
def modifySharpToNumber(inputList):
    outputList = list();
    for line in inputList:
        if line.find("#") > -1 : line = line.replace("#", "Number")
        outputList.append(line)
    return outputList

In [ ]:
def modifySpToSpecial(inputList):
    outputList = list();
    for line in inputList:
        if line.find("Sp") > -1 : 
            line = line.replace("Sp", "Special", 1)
        else :
            line = line.replace("공 마술사", "Technical", 1)
            line = line.replace("빠름", "Quick", 1)
            line = line.replace("헤딩", "Head", 1)
            line = line.replace("힘 있음", "Powerful", 1)
            line = line.replace("예측할 수 없음", "Unpredictable", 1)
        outputList.append(line)
    return outputList

In [ ]:
def removeFirstColumn(inputList):
    outputList = list();
    for line in inputList:
        col = line.split(",")
        tmpStr = ""
        
        for i in range(1,len(col)):
            tmpStr += col[i] + ","
        tmpStr = tmpStr[:-1]
        outputList.append(tmpStr)
    return outputList

In [ ]:
def dividePlayerAndPlayerID(inputList):
    outputList = list();
    for line in inputList:
        col = line.split(",")
        tmpStr = ""
        if col[2] == "Player": 
            col[2] = "Player,PlayerID"
            tmpStr += col[0]
            for i in range(1, len(col)):
                tmpStr += "," + col[i]
            outputList.append(tmpStr)
        else : 
            tmpStr = col[2].replace(" [playerid=", ",", 1)
            endIndex = tmpStr.find("]")
            col[2] = tmpStr[:endIndex]
            
            tmpStr = col[0]
            for i in range(1, len(col)):
                tmpStr += "," + col[i]
            outputList.append(tmpStr)
    return outputList

In [ ]:
from datetime import datetime
from dateutil.relativedelta import relativedelta
import re

In [ ]:
class SeasonWeekDay:
    
    def __init__(self, timeStr):
        for myStr in timeStr.split(' '):
            if ( re.search('시즌', myStr) ):
                self.seasonInt = int(myStr[:-2])
            if ( re.search('주', myStr) ):
                self.weekInt = int(myStr[:-1])
            if ( re.search('일', myStr) ):
                self.dayInt = int(myStr[:-1])
    
    def display(self):
        outStr = ''
        if (hasattr(self, 'seasonInt')):
            if (self.seasonInt > 0):
                outStr += str(self.seasonInt) + '시즌 '
        if (hasattr(self, 'weekInt')):
            outStr += str(self.weekInt) + '주 '
        if (hasattr(self, 'dayInt')):
            outStr += str(self.dayInt) + '일'
        return outStr
    
    def modify(self, diffDayInt):
        if (self.dayInt >= diffDayInt):
            self.dayInt = self.dayInt - diffDayInt
        else :
            if (self.weekInt > 0):
                self.weekInt -= 1
                self.dayInt = self.dayInt - diffDayInt + 7
            else :
                if (self.weekInt == 0):
                    self.seasonInt -= 1
                    self.weekInt = 15
                    self.dayInt = self.dayInt - diffDayInt + 7

In [ ]:
def modifySince(inputList, targetStr, nowStr):   
    outputList = list();
    for line in inputList:
        col = line.split(",")
        if col[7] != "Since":            
            col[7] = col[7].replace(" 시즌", "시즌", 1).replace("시즌 ", "시즌", 1).replace("시즌 ", "시즌", 1)
            col[7] = col[7].replace(" 주", "주", 1).replace("주 ", "주", 1).replace("주 ", "주", 1)
            col[7] = col[7].replace(" 일", "일", 1)
            
            targetDay = datetime.strptime(targetStr, '%Y/%m/%d').date()
            nowDay = datetime.strptime(nowStr, '%Y/%m/%d').date()
            diffDay = relativedelta(nowDay, targetDay)
            
            #print(col[7])
            
            modifySinceStr = SeasonWeekDay(col[7])
            modifySinceStr.modify(int(diffDay.days))
            
            #print(int(diffDay.days), col[7], modifySinceStr.display())
            col[7] = modifySinceStr.display()
            
            tmpStr = col[0]
            for i in range(1, len(col)):
                tmpStr += "," + col[i]
            
            outputList.append(tmpStr)
        else:
            outputList.append(line)
    return outputList

In [ ]:
def modifyMB(inputList):
    outputList = list();
    for line in inputList:
        col = line.split(",")
        if col[14] == "MB":
            outputList.append(line)
        if len( col[14] ) == 1:
            tmpLine = line.replace("✔", "TRUE", 1)
            outputList.append(tmpLine)
        if len( col[14] ) == 0:
            col[14] = "FALSE"
            tmpLine = col[0]
            for i in range(1, len(col) ):
                tmpLine += "," + col[i]
            outputList.append(tmpLine)
    return outputList

In [ ]:
def modifyLast(inputList):
    outputList = list();
    for line in inputList:
        col = line.split(",")
        if col[23] == "Last":
            outputList.append(line)
        else :
            col[23] = col[23][:10]
            tmpLine = col[0]
            for i in range(1, len(col)):
                tmpLine += "," + col[i]
            outputList.append(tmpLine)
    return outputList

In [ ]:
def assignEmptyForTC(inputList):
    outputList = list();
    for line in inputList:
        col = line.split(",")
        if col[28] == "TC":
            outputList.append(line)
        else :
            col[28] = ""
            tmpLine = col[0]
            for i in range(1, len(col)):
                tmpLine += "," + col[i]
            outputList.append(tmpLine)
    return outputList

In [ ]:
def assignEmptyForPH(inputList):
    outputList = list();
    for line in inputList:
        col = line.split(",")
        if col[29] == "PH":
            outputList.append(line)
        else :
            col[29] = ""
            tmpLine = col[0]
            for i in range(1, len(col)):
                tmpLine += "," + col[i]
            outputList.append(tmpLine)
    return outputList

In [ ]:
def modifyKPPos(inputList):
    outputList = list();
    for line in inputList:
        col = line.split(",")
        if col[30] == "KP": 
            col[30] = "KPPos"
            tmpLine =col[0]
            for i in range(1, len(col)):
                tmpLine += "," + col[i]
            outputList.append(tmpLine)
        else :
            outputList.append(line)
    return outputList

In [ ]:
def modifyWBPos(inputList):
    outputList = list();
    for line in inputList:
        col = line.split(",")
        if col[31] == "WB": 
            col[31] = "WBPos"
            tmpLine =col[0]
            for i in range(1, len(col)):
                tmpLine += "," + col[i]
            outputList.append(tmpLine)
        else :
            outputList.append(line)
    return outputList

In [ ]:
def modifyCDPos(inputList):
    outputList = list();
    for line in inputList:
        col = line.split(",")
        if col[32] == "CD": 
            col[32] = "CDPos"
            tmpLine =col[0]
            for i in range(1, len(col)):
                tmpLine += "," + col[i]
            outputList.append(tmpLine)
        else :
            outputList.append(line)
    return outputList

In [ ]:
def modifyWPos(inputList):
    outputList = list();
    for line in inputList:
        col = line.split(",")
        if col[33] == "W": 
            col[33] = "WPos"
            tmpLine =col[0]
            for i in range(1, len(col)):
                tmpLine += "," + col[i]
            outputList.append(tmpLine)
        else :
            outputList.append(line)
    return outputList

In [ ]:
def modifyIMPos(inputList):
    outputList = list();
    for line in inputList:
        col = line.split(",")
        if col[34] == "IM": 
            col[34] = "IMPos"
            tmpLine =col[0]
            for i in range(1, len(col)):
                tmpLine += "," + col[i]
            outputList.append(tmpLine)
        else :
            outputList.append(line)
    return outputList

In [ ]:
def modifyFWPos(inputList):
    outputList = list();
    for line in inputList:
        col = line.split(",")
        if col[35] == "FW": 
            col[35] = "FWPos"
            tmpLine =col[0]
            for i in range(1, len(col)):
                tmpLine += "," + col[i]
            outputList.append(tmpLine)
        else :
            outputList.append(line)
    return outputList

In [ ]:
def removeLastString(inputList):
    outputList = list();
    for line in inputList:
        outputList.append( line[0:-1] )
    return outputList

In [ ]:
import csv
lineList = [] # emptyList

with open(folder + '/player.html', 'r', encoding='utf-8') as f:
    reader = csv.reader(f, delimiter='\n')
    for row in reader:
        #print(row[0])
        lineList.append(row[0])
lineList

In [ ]:
#lineList = tmpClip.split("\n");

lineList = removeHattrickTag(lineList)
lineList = combineTwoLine(lineList)
lineList = modifySharpToNumber(lineList)
lineList = modifySpToSpecial(lineList)
lineList = removeFirstColumn(lineList)
lineList = dividePlayerAndPlayerID(lineList)
lineList = modifySince(lineList, folder, nowDate.replace('-','/'))
lineList = modifyMB(lineList)
lineList = modifyLast(lineList)
lineList = assignEmptyForTC(lineList)
lineList = assignEmptyForPH(lineList)
lineList = modifyKPPos(lineList)
lineList = modifyWBPos(lineList)
lineList = modifyCDPos(lineList)
lineList = modifyWPos(lineList)
lineList = modifyIMPos(lineList)
lineList = modifyFWPos(lineList)
lineList = removeLastString(lineList)

In [ ]:
for line in lineList:
    col = line.split(",")
    print(len(col), col[0], sep="\t" ) # 36

In [ ]:
import os.path

filePath = folder + '/player.html'
if ( os.path.exists(filePath) ):
    print(filePath + "\t : will be removed")
    os.remove(filePath)

filePath = folder + '/player.txt'
if not os.path.exists(folder):
    print(folder + "\t : will be created")
    os.makedirs(folder)

if ( os.path.exists(filePath) ):
    print(filePath + "\t : will be removed")
    os.remove(filePath)
    
if not os.path.exists(filePath):
    print(filePath + "\t : is not exists")

In [ ]:
with open(folder+'/player.txt', 'a') as f:
    for line in lineList:
        f.write(line + "\n")

In [ ]:
# player_skill_table from file to table

In [ ]:
#import pymsgbox

In [ ]:
#pymsgbox.alert(text='', title='', button='OK')

In [ ]:
#getText = pymsgbox.confirm(text='Choose OK or Cancel', title='', buttons=['OK', 'Cancel'])

In [ ]:
#print(getText)

In [ ]:
import psycopg2

In [ ]:
def dropPlayerTable():
    conn = None
    try:
        conn = psycopg2.connect("dbname='mydatabase2' user='myuser' host='localhost' port='65432' password='123qwe'")
        cur = conn.cursor()
        sql = """DROP TABLE IF EXISTS player ; """
        cur.execute(sql)
        conn.commit()
    except(Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()

In [ ]:
#dropPlayerTable()

In [ ]:
def createPlayerTable():
    conn = None
    try:
        conn = psycopg2.connect("dbname='mydatabase2' user='myuser' host='localhost' port='65432' password='123qwe'")
        cur = conn.cursor()
        sql = """
CREATE TABLE player (
    Date DATE,           --01
    Num int,             --02
    Nat VARCHAR(30),     --03
    Player VARCHAR(30),  --04
    PlayerID BIGINT,     --05
    Spacial VARCHAR(30), --06
    st VARCHAR(10),      --07  Status Yellow, Red
    Age VARCHAR(10),     --08
    Since VARCHAR(30),   --09
    TSI BIGINT,          --10
    LS INT,              --11
    XP INT,              --12
    Fo INT,              --13
    Stm INT,             --14
    Lo INT,              --15
    MB boolean,          --16
    KP INT,              --17
    DF INT,              --18
    PM INT,              --19
    WI INT,              --20
    PS INT,              --21
    SC INT,              --22
    SP INT,              --23
    con BIGINT,          --24 Psico
    Last DATE,           --25
    RT NUMERIC(2, 1),    --26 Rating
    Po VARCHAR(5),       --27 Position
    Wage BIGINT,         --28
    G INT,               --29
    --TC VARCHAR(1),     --
    --PH VARCHAR(1),     --
    KP_P NUMERIC(4, 2),  --30 KP Position
    WB_P NUMERIC(4, 2),  --31 WB Position
    CD_P NUMERIC(4, 2),  --32 CD Position
    W_P NUMERIC(4, 2),   --33 W  Position
    IM_P NUMERIC(4, 2),  --34 IM Position
    FW_P NUMERIC(4, 2)   --35 FW Position
) ; """
        cur.execute(sql)
        conn.commit()
    except(Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()

In [ ]:
#createPlayerTable()

In [ ]:
def insertPlayerTable(time, row):
    conn = None
    try:
        conn = psycopg2.connect("dbname='mydatabase2' user='myuser' host='localhost' port='65432' password='123qwe'")
        cur = conn.cursor()
        sql =  """
INSERT INTO player ( 
    SELECT 
        to_date(%s, 'YYYY-MM-DD'),                                    -- 1
        %s, %s, %s, %s, %s,                                           -- 2, 3, 4, 5, 6
        %s, %s, %s, %s, %s,                                           -- 7, 8, 9,10,11
        %s, %s, %s, %s, %s,                                           --12,13,14,15,16
        %s, %s, %s, %s, %s,                                           --17,18,19,20,21
        %s, %s, %s,                                                   --22,23,24
        to_date(coalesce(nullif(%s,''), '0001-01-01'), 'YYYY-MM-DD'), --25
        cast(coalesce(nullif(%s,''),'0.0') as float),                 --26
        %s, %s, %s,                                                   --27,28,29
        %s, %s, %s, %s, %s,                                           --30,31,32,33,34
        %s                                                            --35
) ; """
        #print(sql + "\t")
        cur.execute(sql, ( time, 
                           row[0], row[1], row[2], row[3], row[4],
                           row[5], row[6], row[7], row[8], row[9],
                           row[10],row[11],row[12],row[13],row[14], 
                           row[15],row[16],row[17],row[18],row[19],
                           row[20],row[21],row[22],
                           row[23],
                           row[24],
                           row[25],row[26],row[27],
                           row[30],row[31],row[32],row[33],row[34],
                           row[35]
                         )
                   )
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print(sql)
        print(row)
        print("Error Happend")
        print(error)
    finally:
        if conn is not None:
            conn.close()

In [ ]:
import csv
with open(folder + '/player.txt', 'r', encoding='utf-8') as f:
    reader = csv.reader(f, delimiter=',')
    for row in reader:
        if row[0] != "Number" :
            insertPlayerTable( folder.replace('/','-'), row)

In [ ]:
def selectPlayer(SelectString, WhereString):
    outputList = list();
    sql = ""
    conn = None
    try:
        conn = psycopg2.connect("dbname='mydatabase2' user='myuser' host='localhost' port='65432' password='123qwe'")
        cur = conn.cursor()
        sql = SelectString + "\nFROM player" + WhereString + "\n"
        print(sql)
        
        cur.execute(sql)
        row = cur.fetchone()
        while row is not None:
            #print(row)
            outputList.append(row)
            row = cur.fetchone()
    except (Exception, psycopg2.DatabaseError) as error:
            print(error)
            print(sql)
    finally:
        if conn is not None:
            conn.close()
    return outputList

In [ ]:
SelectString = ""
SelectString += "\n" + "SELECT "
SelectString += "\n" + "    date :: VARCHAR(10), num, player, spacial, st,                       --01 05 "
SelectString += "\n" + "    age, since, tsi, ls, xp,                                             --06 10 "
SelectString += "\n" + "    fo, stm, lo, mb, kp,                                                 --11 15 "
SelectString += "\n" + "    df, pm, wi, ps, sc,                                                  --16 20 "
SelectString += "\n" + "    sp, con, last :: VARCHAR(10), rt :: VARCHAR(3), po,                  --21 25 "
SelectString += "\n" + "    wage, g, kp_p :: VARCHAR(5), wb_p :: VARCHAR(5), cd_p :: VARCHAR(5), --26 30 "
SelectString += "\n" + "    w_p :: VARCHAR(5), im_p :: VARCHAR(5), fw_p :: VARCHAR(5)            --31 35 "

WhereString = ""
WhereString += "\n" + "WHERE "
WhereString += "\n" + "    date = last "
WhereString += "\n" + "    and date = '" + folder.replace('/','-') + "' "
WhereString += "\n" + "    order by num,date "

tupleList = selectPlayer(SelectString, WhereString)
for tuple in tupleList:
    print(tuple[0], tuple[1], tuple[2], tuple[3], tuple[4],
          tuple[5], tuple[6], tuple[7], tuple[8], tuple[9],
          tuple[10], tuple[11], tuple[12], tuple[13], tuple[14],
          tuple[15], tuple[16], tuple[17], tuple[18], tuple[19],
          tuple[20], tuple[21], tuple[22], tuple[23], tuple[24],
          tuple[25], tuple[26], tuple[27], tuple[28], tuple[29],
          tuple[30], tuple[31], tuple[32],
          sep=',')

In [ ]:
SelectString = ""
SelectString += "\n" + "SELECT "
SelectString += "\n" + "    num, playerid --01 02 "

WhereString = ""
WhereString += "\n" + "WHERE "
WhereString += "\n" + "    date = '" + folder.replace('/','-') + "' "
WhereString += "\n" + "    order by num "

numPlayerIDtupleList = selectPlayer(SelectString, WhereString)
for tuple in numPlayerIDtupleList:
    print(tuple[0], tuple[1], sep="\t")

In [ ]:
import os.path
filePath = folder + '/numPlayerID.txt'

if ( os.path.exists(filePath) ):
    os.remove(filePath)
    print(filePath + " <- this file is deleted.")

if not ( os.path.exists(filePath) ):
    print(filePath + " <- this file is not exists.")

In [ ]:
with open (folder + '/numPlayerID.txt', 'a') as f:
    f.write( '\n'.join('%s,%s' % myTuple for myTuple in numPlayerIDtupleList) + '\n' )